## 1. 패키지 설치
Retrieval Augmented Generation(RAG) pipeline 구성을 위해 필요한 패키지 설치

In [1]:
!pip install langchain-chroma datasets


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='nlpai-lab/KURE-v1')

In [3]:
from langchain_chroma import Chroma
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

In [4]:
train, validation = dataset['train'], dataset['validation']

In [5]:
from langchain.schema import Document
from datasets import Dataset


def squad_to_document(ds):
    docs = []
    for item in ds:
        metadata = {
            "id": item["id"],
            "title": item["title"],
            "question": item["question"],
            "answers": item["answers"]["text"][0] if item["answers"]["text"] else "",
        }
        docs.append(Document(page_content=item["context"], metadata=metadata))
    return docs


def dedupe_contexts(ds):
    seen = set()
    filtered = []

    for item in ds:
        ctx = item.get('context')
        if ctx not in seen:
            seen.add(ctx)
            filtered.append(item)

    if isinstance(ds, Dataset):
        # Dataset으로 다시 만들어 반환
        return Dataset.from_list(filtered)
    else:
        return filtered

In [6]:
deduped_list = dedupe_contexts(train)

In [7]:
print(deduped_list[5])

{'id': '6561639-2-0', 'title': '커닐링구스', 'context': '성경의 아가서 7 : 3 (제임스 왕 역에서 7 : 2)는 커닐링구스에 대한 우회적 언급이 포함되어 있다고도 생각할 수 있지만, 많은 번역자는 핵심 단어를 "배꼽"로 번역하고 있다. 일부는 "여성의 웅덩이에 포도주가 가득 채워지고 있다는 묘사는 육욕적인 그릇에서 마시고 싶은 남자의 욕망을 암시하고 있다. 따라서 이것은 교정의 미묘하고 깊은 맛의 귀띔인지도 모른다"라는 해석을 보여주고 있다. 다른 번역에서는 다음과 같이도 읽을 수있는 - "너의 음부는 동그란 컵, 칵테일이 끊어진 적이 없다."(아가 7 : 2 שררך אגן הסהר אל יחסר המזג) 샌들에서 출발 "vulva"(히브리어 shor-- 아람어 \'비밀 장소\'라는 단어에서 파생 된), 배, 가슴으로 올라 간다고 하는 문맥이 단어의 의미를 거의 결정 짓는 것이다. 기독교와 유대교의 여러 전통에서 "아가"에 그려진 신랑과 신부의 에로틱한 친밀감에 영적인 의미를 부여하고 있다.', 'question': '성경에서 커닐링구스에 대해 우회적으로 번역한 단어는 무엇인가?', 'answers': {'answer_start': [88], 'text': ['배꼽']}}


In [8]:
document_list = squad_to_document(deduped_list)

In [9]:
len(document_list)

9606

In [13]:
from langchain_core.documents import Document
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from typing import List


def batch_add_documents(
        pinecon_store: PineconeVectorStore,
        documents: List[Document],
        batch_size: int = 10,
) -> None:
    idx = 0
    total_docs = len(documents)

    while idx < total_docs:
        end_idx = min(idx + batch_size, total_docs)
        batch = documents[idx: end_idx]
        try:
            pinecon_store.add_documents(batch)
            print(f"{idx + 1} ~ {end_idx}")
        except Exception as e:
            print(e)
        idx = end_idx

In [21]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

index_name = '~~~~'  ## 실제 index name
pc = Pinecone(api_key="~~~~")  ## 실제 api key 으로 대체할 것!!!!!!

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),  # 자유
    )
index = pc.Index(index_name)

vectorstore = PineconeVectorStore(index=index, embedding=embeddings)
batch_add_documents(vectorstore, document_list)

1 ~ 10
11 ~ 20
21 ~ 30
31 ~ 40
41 ~ 50
51 ~ 60
61 ~ 70
71 ~ 80
81 ~ 90
91 ~ 100
101 ~ 110
111 ~ 120
121 ~ 130
131 ~ 140
141 ~ 150
151 ~ 160
161 ~ 170
171 ~ 180
181 ~ 190
191 ~ 200
201 ~ 210
211 ~ 220
221 ~ 230
231 ~ 240
241 ~ 250
251 ~ 260
261 ~ 270
271 ~ 280
281 ~ 290
291 ~ 300
301 ~ 310
311 ~ 320
321 ~ 330
331 ~ 340
341 ~ 350
351 ~ 360
361 ~ 370
371 ~ 380
381 ~ 390
391 ~ 400
401 ~ 410
411 ~ 420
421 ~ 430
431 ~ 440
441 ~ 450
451 ~ 460
461 ~ 470
471 ~ 480
481 ~ 490
491 ~ 500
501 ~ 510
511 ~ 520
521 ~ 530
531 ~ 540
541 ~ 550
551 ~ 560
561 ~ 570
571 ~ 580
581 ~ 590
591 ~ 600
601 ~ 610
611 ~ 620
621 ~ 630
631 ~ 640
641 ~ 650
651 ~ 660
661 ~ 670
671 ~ 680
681 ~ 690
691 ~ 700
701 ~ 710
711 ~ 720
721 ~ 730
731 ~ 740
741 ~ 750
751 ~ 760
761 ~ 770
771 ~ 780
781 ~ 790
791 ~ 800
801 ~ 810
811 ~ 820
821 ~ 830
831 ~ 840
841 ~ 850
851 ~ 860
861 ~ 870
871 ~ 880
881 ~ 890
891 ~ 900
901 ~ 910
911 ~ 920
921 ~ 930
931 ~ 940
941 ~ 950
951 ~ 960
961 ~ 970
971 ~ 980
981 ~ 990
991 ~ 1000
1001 ~ 1010
1011 ~ 

In [36]:
vectorstore.add_documents(document_list[5358:5360])
# gpu ram issue로 이 문서만 수동 추가해주었음.

['b319b0fd-2842-4435-8cc1-571348fa5e35',
 'ec297800-0ac8-405c-bf3f-352eb7b95f3b']